In [1]:
import os
from pathlib import Path
import random

import datetime as dt
import itertools as it

from collections import namedtuple

from tqdm.notebook import tqdm # Progress bars

# https://import-as.github.io
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sklearn as sk
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import src
from src.data import get_df, filter_df

POSSIBLE_PATHS = ["./datawarehouse", "/kaggle/input/dao-analyzer"]

DW = None
for p in POSSIBLE_PATHS:
    DW = Path(p)
    if DW.is_dir():
        break
else:
    print("No se ha encontrado el DW")

src.data.DEFAULT_PATH = DW

Hyperparameters table in [Google Drive](https://docs.google.com/spreadsheets/d/1riafpWt1563w9pbqdt1g2QZVkc7TfRWGzFaCG5rudDI/edit?usp=sharing)

In [2]:
# Remove users with less than 6 votes from the dataset before splitting
MIN_VOTES = 6
TESTING_DAO_NAMES = {'dxDAO', 'xDXdao'}
K_FOLD = 5

EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
DECAY = 0.0001
# Latent space dimensions
EMBEDDING_DIM = 32
CONV_LAYERS = 3

# Reading data

In [3]:
def load_data():
    dfv = filter_df(get_df('votes'))

    # Convert to categories
    CAT_COLS = ['voter', 'dao', 'proposal']
    dfv[CAT_COLS] = dfv[CAT_COLS].astype('category')

    # Remove the ones with less than MIN_VOTES votes
    vpu = dfv.groupby('voter').size()
    dfv = dfv[dfv['voter'].isin(vpu[vpu >= MIN_VOTES].index)]
    
    return dfv

dfv = load_data()
dfv

,network,createdAt,voter,outcome,reputation,dao,proposal
id,,,,,,,
0x000682a038b22925343bd5b9e84acb424a9d148843261372dc83514589bfdc3b,mainnet,2023-03-14 02:47:23,0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501,Pass,90248500000000000000000,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb2a2b6baeed346c4e49a241044bd547504d02a3b593a...
0x001cde4d98e26191fc34308be3b51ce50d3eb9561b1b01b768722bf9abe04457,mainnet,2022-05-22 02:09:49,0xabd238fa6b6042438fbd22e7d398199080b4224c,Pass,68359641109260095429647,0x519b70055af55a007110b4ff99b0ea33071c720a,0x53c9666338692a720a3ee3841b49f58a4d580cf7ed11...
0x003b53eeeb314ab7fa42f88f8630d7fd6ea17184c1f05a68abba789c881b9119,mainnet,2020-08-28 12:12:34,0xb0e83c2d71a991017e0116d58c5765abc57384af,Pass,18016183440825327276060,0x519b70055af55a007110b4ff99b0ea33071c720a,0xdd9f7b1cbd148a266c6a6edbcf271ea0248bfa7f9874...
0x003e74b43ea9d833d524b82c47c4e1b2b02f84506bd233382840a5f63add4e3d,mainnet,2023-03-26 15:16:47,0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501,Pass,90248500000000000000000,0x519b70055af55a007110b4ff99b0ea33071c720a,0x2f701be3fbd3e7e706f1aac4d36839c18a56a52e45e5...
0x0057ad41e781a0acd4e7a06be5cea9dc3fcd5fc259ff409b8ed4c504e01cfd8a,mainnet,2022-12-02 20:13:47,0x8e900cf9bd655e34bb610f0ef365d8d476fd7337,Pass,86679381514995439609053,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb7ff31ec2bcdb8a254b8a1b06c6d2080a4089135b03e...
...,...,...,...,...,...,...,...
0xffd8ca7f39b9cf2c8de1914306c1b69e78a958ba90dbd73b0bb6332d3771650e,xdai,2021-05-31 10:15:35,0x6f7c864cc0fc9fb2fe26f53c031d3deec0b8d7d5,Pass,4503667578035201114112,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0x84d7402de697479a586620cda052c2b6311238d8c9a0...
0xffe4fb38b16e8963340aa482be165fb4d650f321975e5f5faa3aa0e9abbf9528,xdai,2021-05-09 07:15:35,0x5a3992044a131c2f633394065c13ba1b33cdffd9,Pass,17609542272000000000000,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0x5f50da5227bbb459f6f5a78be5d2cf0000da5c7ceca1...
0xffe62001e3e32cabf75e660f91057fc7ed13232c3260f4d5e1469259d62975d9,xdai,2021-10-26 18:37:55,0xc4a69fbf4511a1377161834cb7a3b8766953db02,Pass,372907972512994918632,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0xbf65ac67817ace37cc73d62b507336c8adc41454358c...


# Transform data to tensors

In [4]:
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as PyG
from torch_geometric.nn.conv import MessagePassing

/tmp/ipykernel_55775/4095528591.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [129]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import preprocessing as pp

class NodeTransformer(TransformerMixin):
    """ Given a votes DataFrame with voters and proposals
    adds a unique voter id and proposal id """
    def __init__(self):
        super().__init__()
        self.encoder_user = pp.LabelEncoder()
        self.encoder_prop = pp.LabelEncoder()

    def fit(self, X):
        self.encoder_user.fit(X['voter'])
        self.encoder_prop.fit(X['proposal'])
        return self

    def transform(self, X):
        _X = X.copy()
        _X['uid'] = self.encoder_user.transform(X['voter'])
        _X['pid'] = self.encoder_prop.transform(X['proposal']) + self.N_USERS
        return _X

    @property
    def N_USERS(self):
        return len(self.encoder_user.classes_)

    @property
    def N_ITEMS(self):
        return len(self.encoder_prop.classes_)

def get_edge_index(X):
    """ Given a votes DataFrame with uid and pid
    transforms it to a list of links (COO format)
    with shape [2, n_votes*2]
    """
    u_t = torch.LongTensor(X['uid'])
    p_t = torch.LongTensor(X['pid'])

    edge_index = torch.stack([
        torch.cat([u_t, p_t]),
        torch.cat([p_t, u_t]),
    ])

    return edge_index.squeeze()

ntf = NodeTransformer()
dfv = ntf.fit_transform(dfv)

edge_index = get_edge_index(dfv)
assert edge_index.shape == (2, 2*len(dfv))
edge_index.shape

torch.Size([2, 16606])

In [130]:
BatchSample = namedtuple('BatchSample', 'users pos neg')
def batch_sample(df, batch_size: int, n_users: int) -> BatchSample:
    """ Returns a sample of positive and negative indices """
    if batch_size > n_users:
        raise ValueError("Sample size should be smaller than total number of users")
        
    def _sample_neg(x):
        """ Returns an uid that is not in x """
        return random.choice(pd.RangeIndex(n_users).difference(x))
        
    voted = df.groupby('uid')['pid'].apply(list).reset_index()
        
    users = voted.sample(batch_size).index.sort_values()
    users_df = pd.DataFrame(users, columns=['uid'])
    
    voted = pd.merge(voted, users_df, how='right', on=['uid'])
    # in the target class (voted)
    pos = voted['pid'].apply(lambda x: random.choice(x)).values
    # not in the target class (not voted)
    neg = voted['pid'].apply(_sample_neg).values
    
    return BatchSample(
        users=torch.LongTensor(users).to(device),
        # Adding n_users as it is an item index
        pos=torch.LongTensor(pos).to(device) + n_users,
        neg=torch.LongTensor(neg).to(device) + n_users,
    )

batch_sample(dfv, 16, n_users=ntf.N_USERS)

BatchSample(users=tensor([ 5,  8, 24, 36, 40, 43, 67, 68, 74, 77, 78, 79, 83, 90, 95, 96],
       device='cuda:0'), pos=tensor([ 682,  767, 1092, 2001, 1704, 1378, 1121, 1914,  319, 1197, 1069, 1986,
        1999, 1737,  482, 1842], device='cuda:0'), neg=tensor([149, 175, 121, 130, 181, 197, 169, 182, 122, 108, 192, 202, 126, 146,
        105, 166], device='cuda:0'))

# Creating the model

We will implement LightGCN in PyG

In [197]:
from src.models.LightGCN import LightGCN
import skorch

# TODO: Move this to LightGCN.py
class BPRLoss(nn.Module):
    @staticmethod
    def forward(W, u, p, n):
        pos_scores = torch.mul(W[u], W[p])
        neg_scores = torch.mul(W[u], W[n])
        bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

        return bpr_loss

class RegLoss(nn.Module):
    def __init__(self, decay=0.0001):
        self.decay = decay
    
    def forward(self, W0, u, p, n):
        return 1/2 * self.decay * (
            W0[u].norm().pow(2) +
            W0[p].norm().pow(2) +
            W0[n].norm().pow(2)
        ) / float(len(u))

class GraphDataset(skorch.dataset.Dataset):
    def __getitem__(self, i):
        # print('isndframe:', self.X_is_ndframe)
        x, y = super().__getitem__(i)
        return x, y

class SkLightGCN(skorch.NeuralNet):
    def __init__(self, *args, **kwargs):
        super().__init__(
            module=LightGCN,
            optimizer=torch.optim.Adam,
            criterion=BPRLoss,
            dataset=GraphDataset,
            # train_split=skorch.dataset.ValidSplit(5, stratified=True),
            train_split=None,
            *args, **kwargs,
        )

        self.ntf = NodeTransformer()
        self.iterator_train__collate_fn = self.custom_collate
        self.iterator_test__collate_fn = self.custom_collate

    @property
    def N_USERS(self):
        return self.ntf.N_USERS

    @property
    def N_ITEMS(self):
        return self.ntf.N_ITEMS

    def _negative_samples(self, pos_samples):
        return [ x for x in range(self.ntf.N_USERS, self.ntf.N_USERS+self.N_ITEMS) if x not in set(pos_samples) ]

    def custom_collate(self, items):
        u, p, n = [], [], []
        for x, y in items:
            u.append(x['uid'])
            p.append(random.choice(x['pid']))
            n.append(random.choice(self._negative_samples(x['pid'])))

        return BatchSample(u, p, n), [y for x,y in items]

    def fit(self, X, **fit_params):
        """ The target is predicting if there is a link between X and Y.
        Therefore, we will pass an X with the voters and proposals, and
        we will try to create the index from that.
        """

        # Transforming every user and item to its node id
        if not self.initialized_:
            X = self.ntf.fit_transform(X)
    
            self.module__n_users = self.ntf.N_USERS
            self.module__n_items = self.ntf.N_ITEMS

            if self.batch_size > self.ntf.N_USERS:
                print("Warning! Batch size greater than the number of users")
        else:
            X = self.ntf.transform(X)

        X = X.groupby('uid')['pid'].apply(torch.LongTensor).reset_index()
        # X = X[['uid', 'pid']]
        super().fit(X, **fit_params)
    
    def infer(self, x, **fit_params):
        u, p, n = x
        print('infer:', len(x), len(u), len(p), len(n))
        # The module receives the edge_index, but the loss uses
        # positive/negative sampling
        return self.module_(x, **fit_params)

    # TODO: Important! Change the _batcher_ so it also returns the positive and negative samples
    # the positive and negative samples were 
    def get_iterator(self, dataset, training=False):
        return super().get_iterator(dataset, training)

    def get_loss(self, y_pred, y_true, X=None, training=False):
        """
        y_pred : torch tensor
            The embeddings of size [N_USERS+N_ITEMS, EMBEDDING_SIZE]
        y_true : torch tensor
            Dummy zeroes vector (None)
        X : input data (the batch of size `batch_size`)
        """
        print(self.iterator_train)
        assert isinstance(X, BatchSample)
        u, p, n = X
        bpr = BPRLoss.forward(y_pred, u, p, n)
    
skmodel = SkLightGCN(batch_size=32, device=device)
skmodel.fit(dfv)

infer: 3 32 32 32


ValueError: too many values to unpack (expected 2)

In [17]:
def _assert_eq(x, y):
    import sys
    if x != y:
         print(f"Expected {x}, got {y}", file=sys.stderr)

class Metrics:
    def __init__(self, model, Ks=[3,6,12]):
        super().__init__()
        self.model = model
        
        if isinstance(Ks, int):
            self.Ks = [Ks]
        else:
            self.Ks = Ks
        
        self.epochs = 0
        self.batches = 0
        self.precision = {k:[] for k in self.Ks}
        self.recall = {k:[] for k in self.Ks}
        self.train_precision = {k:[] for k in self.Ks}
        self.train_recall = {k:[] for k in self.Ks}
        self.bpr_loss = []
        self.reg_loss = []
        self._batch_bpr_loss = []
        self._batch_reg_loss = []
    
    def update(self):
        out = model(edge_index)
        user_embed, item_embed = torch.split(out, (self.model.n_users, self.model.n_items))
        
        ### Metrics
        relevance_score = torch.matmul(user_embed, torch.transpose(item_embed, 0, 1))
        # get all user-item interactions (ground truth)
        i = torch.stack([
            torch.LongTensor(train_df['uid'].values),
            torch.LongTensor(train_df['pid'].values),
        ])
        v = torch.ones(len(train_df), dtype=torch.float64)
        t_interactions = torch.sparse.FloatTensor(i, v, (self.model.n_users, self.model.n_items)).to_dense().to(device)
        # mask out training user-item interactions from metric computation
        # We are only interested in novel items, as a user won't be interested
        # in "voting again"
        msk_relevance_score = torch.mul(relevance_score, (1 - t_interactions))
        
        # Nevertheless, to test training accuracy, we will check if the suggested interactions
        # are correct, even if they already appeared
        
        train_interacted_items = train_df.groupby('uid')['pid'].apply(list).reset_index()
        test_interacted_items  =  test_df.groupby('uid')['pid'].apply(list).reset_index()
        
        for K in self.Ks:
            topk_relevance_indices_msk = torch.topk(msk_relevance_score, K).indices
            topk_relevance_indices = torch.topk(relevance_score, K).indices
            
            def _userPrecRec(user, scores):
                # assert len(topk_relevance_indices[user['uid']]) == K
                
                # The .tolist() was more important than I thought T.T
                common = set(user['pid']).intersection(scores[user['uid']].tolist())
                # print("##########")
                # print(user['uid'])
                # print(user['pid'])
                # print(topk_relevance_indices[user['uid']])
                ncommon = len(common)
                
                # user['precision'] = ncommon / K
                user['precision'] = ncommon / min(K, len(user['pid']))
                user['recall'] = ncommon / len(user['pid'])
                
                return user
            
            train_interacted_k = train_interacted_items.apply(_userPrecRec, args=(topk_relevance_indices,), axis=1)
            interacted_k = test_interacted_items.apply(_userPrecRec, args=(topk_relevance_indices_msk,), axis=1)
            
            self.train_precision[K].append(train_interacted_k['precision'].mean())
            self.train_recall[K].append(train_interacted_k['recall'].mean())
            self.precision[K].append(interacted_k['precision'].mean())
            self.recall[K].append(interacted_k['recall'].mean())
        
        self.bpr_loss.append(np.mean(self._batch_bpr_loss))
        self.reg_loss.append(np.mean(self._batch_reg_loss))
        self._batch_bpr_loss = []
        self._batch_reg_loss = []
        
        self.epochs += 1
        
    def compute_loss(self, users, pos_items, neg_items):
        # bpr loss = -log(sigmoid(score_neg - score_pos))
        out = model(edge_index)
        emb0 = model.embedding.weight
        
        users_emb = out[users]
        pos_emb = out[pos_items]
        neg_emb = out[neg_items]
        userEmb0 = emb[users]
        posEmb0 = emb[pos_items]
        negEmb0 = emb[neg_items]
        
        # compute loss from initial embeddings, used for regulization
        reg_loss = (1 / 2) * (
            userEmb0.norm().pow(2) + 
            posEmb0.norm().pow(2)  +
            negEmb0.norm().pow(2)
        ) / float(len(users))

        # compute BPR loss from user, positive item, and negative item embeddings
        pos_scores = torch.mul(users_emb, pos_emb).sum(dim=1)
        neg_scores = torch.mul(users_emb, neg_emb).sum(dim=1)

        bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))
        
        reg_loss = DECAY * reg_loss
        final_loss = bpr_loss + reg_loss
        
        final_loss.backward()
        
        self._batch_bpr_loss.append(float(bpr_loss))
        self._batch_reg_loss.append(float(reg_loss))
        
        self.batches += 1
        
    def plot_loss(self):
        fig = px.line(metrics.bpr_loss, title=f'BPR loss of model', log_y=True)
        fig.update_layout(
            xaxis_title='epoch',
            yaxis_title='loss',
        )
        return fig
    
    def plot_precision(self, recall=False, train=False):
        df = pd.DataFrame(self.precision).rename(columns='precision@{}'.format)
        
        fig = px.line(df)
        fig.update_layout(
            xaxis_title='epochs',
            yaxis_title='value',
        )
        
        colors = [d.line.color for d in fig.data]
        
        if recall:
            for c, (k,v) in zip(colors, self.recall.items()):
                fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=f'recall@{k}', mode='lines', line={'dash':'dash', 'color': c}))
        
        if train:
            for c, (k,v) in zip(colors, self.train_precision.items()):
                fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=f'ptrain@{k}', mode='lines', line={'dash':'dot'}))
        
        return fig
    
    def plot_recall(self):
        df = pd.DataFrame(self.recall).rename(columns='recall@{}'.format)
        fig = px.line(df)
        fig.update_layout(
            xaxis_title='epochs',
            yaxis_title='value',
        )
        return fig
    
    def __repr__(self):
        return str(self.__dict__)

In [9]:
def batcher(df, batch_size):
    return (df[pos:pos+batch_size] for pos in range(0, len(df), batch_size))

def train_eval(model, optimizer, train_df):
    metrics = Metrics(model)
    
    try:
        for epoch in trange(EPOCHS):
            model.train()
            for batch in batcher(train_df, BATCH_SIZE):
                optimizer.zero_grad()
                users, pos_items, neg_items = batch_sample(train_df, BATCH_SIZE, model.n_users, model.n_items)

                # Inside this, there is the loss.backwards() method
                # Perhaps we should uncouple metrics and loss calculation
                # even though we store the loss in an array to print it later
                metrics.compute_loss(users, pos_items, neg_items)
                optimizer.step()

            model.eval()
            with torch.no_grad():
                metrics.update()
                print(metrics.bpr_loss[-1], metrics.train_precision[6][-1], metrics.precision[6][-1])
    except KeyboardInterrupt:
        pass
        
    return metrics
        
metrics = train_eval(model, optimizer, train_df)
display(metrics.plot_loss())
display(metrics.plot_precision(train=True))

NameError: name 'train_df' is not defined

# Using all of this

Crearé una función que reciba una dirección de un usuario y retorne k propuestas que puedan interesarle

In [19]:
def recommend(user: str, K: int = 12, ignore_train: bool=False):
    uid = encoder_user.transform([user])[0]
    print(f"Recommending {K} proposals for user {user} (uid:{uid}) with {vpu.at[user]} votes")
    
    # Getting embedding
    out = model(edge_index)
    user_embed, item_embed = torch.split(out, (model.n_users, model.n_items))
    relevance_score = torch.matmul(user_embed, torch.transpose(item_embed, 0, 1))
    if ignore_train:
        i = torch.stack([
            torch.LongTensor(train_df['uid'].values),
            torch.LongTensor(train_df['pid'].values),
        ])
        v = torch.ones(len(train_df), dtype=torch.float64)
        t_interactions = torch.sparse.FloatTensor(i, v, (model.n_users, model.n_items)).to_dense().to(device)
        # mask out training user-item interactions from metric computation
        # We are only interested in novel items, as a user won't be interested
        # in "voting again"
        relevance_score = torch.mul(relevance_score, (1 - t_interactions))
    
    topk_relevance_indices = torch.topk(relevance_score, K).indices
    
    pids = topk_relevance_indices[uid].tolist()
    proposals = dfp.loc[encoder_prop.inverse_transform(pids)]
    
    proposals['userVoted'] = dfv.groupby('proposal')['voter'].apply(lambda x: user in set(x))
    
    print(f"precision@{K}={sum(proposals['userVoted'])/len(proposals)*100:.2f}%")
    
    return proposals

user = "0x334f12afb7d8740868be04719639616533075234" # vpu[(12 < vpu) & (vpu < 38)].sample().index[0]
recommend(user, ignore_train=True)[['network', 'createdAt', 'title', 'description', 'userVoted']]

Recommending 12 proposals for user 0x334f12afb7d8740868be04719639616533075234 (uid:18) with 35 votes
precision@12=16.67%


,network,createdAt,title,description,userVoted
id,,,,,
0x52e92a9757df27398403eeced20ee15ae0d5f41f7ea0f956d728d62a7b3f07c5,mainnet,2019-09-16 17:40:16,BC-DAPP Milestones 1 & 2 (dOrg Payout),BC-DAPP Proposal\nThis proposal is to signal a...,False
0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbff9e95873d82c0314534e,mainnet,2019-07-14 14:48:30,Give the power to me,I will make the dxDAO great again!,False
0x59fc60a67b7e89175815610632b9f660a4bbb1e6f40f30dc64590abb5bd7af5e,mainnet,2019-09-16 17:50:28,BC-DAPP Milestones 1 & 2 (Corkus payout),BC-DAPP Proposal\nThis proposal is to signal a...,False
0xb53bf37ddc953a6b764c7b6fd6b058765b9494a17dffc92068229c746eb1cf72,mainnet,2019-07-25 18:32:21,De-whitelist POLY from MGN generating tokens,I am offering de-whitelist POLY from MGN gener...,False
0xb804dbb6ba463b704c5be53c3be122e0f9fd72ebd1d952eb51e74db99a2d7f78,mainnet,2019-07-26 17:47:12,Support Registering dxdao.eth with ENS,dxDAO requires a domain name on a decentralize...,False
0x2bc422bab43d296c6494dcd78bc28a3b10074de875817339efa569af4c383441,mainnet,2019-07-25 17:18:45,Whitelist Compound's cDAI,cDAI is compound's tokenized interest bearing ...,True
0xc343060bc1a1d6669d24875f0998e657ed1ef57cae42edec3971ec657c755324,mainnet,2019-07-31 04:55:17,Support Registering dutchx.eth with ENS,To deploy the DutchX in a fully decentralized ...,False
0x74cbda473a10059324404c16da7ebfb95926da34354607fc89ae2629942e2a24,mainnet,2019-08-13 18:18:06,Add Wrapped BTC - WBTC to whitelist for MGN ge...,Add Wrapped BTC - WBTC to whitelist for MGN ge...,True
0xeb9cf2b3d76664dc1e983137f33b2400ad11966b1d79399d7ca55c25ad6283fa,mainnet,2020-04-17 19:08:38,Finalizing Fundraising Configuration,"Hello community,\n\nThis signal proposal ratif...",False


In [20]:
dfv[dfv['proposal'] == '0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbff9e95873d82c0314534e']

,network,createdAt,voter,outcome,reputation,dao,proposal,random
id,,,,,,,,
0x474e35da8c23aa146bb34356f22e5ca84a56e625b21d02fcee44279990fe4510,mainnet,2019-08-07 19:32:50,0x93d29542401c00f1431fd1c80b634697e5645c59,Fail,2281650741463878942454,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.982379
0x4a745d3cc92c6dacb724dae10b2752123023bd5f52ca417165f69d61c8d878b0,mainnet,2019-08-18 08:35:19,0x3dad32f81f5dc35d961d3da77d32a0a268b8db44,Fail,922534080553417311929,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.809874
0x66d5e3bc30b94b8f8a97b33d6ee20571d0b22b5577692b3e0b8c6c2ae4cfa11c,mainnet,2019-07-15 13:37:44,0x730fd267ef60b27615324b94bf0bc7ed15d52718,Fail,74703181293782997401422,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.819102
0xb6d9cd3f7da07e1bb01f4cf1b0caa3bf714fa86f67bbcf72778c15d9fcf9c652,mainnet,2019-07-17 10:04:32,0x3efd3391a0601eaa093647f911c653d77c11e3fd,Fail,183008951067523277033,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.239180
0xbf412f940e987ea402786e576e194a16818f40deee6c11c79516b3970cb560ba,mainnet,2019-08-09 00:15:49,0x6dd5f1bf1ffa6a3173e333c1588f4cdde8c6799e,Fail,664270618815092643957,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.680228
0xc178fee0b0a49fa090b05b908c326e6ae52f2585c09b2ba353138935dbce8a01,mainnet,2019-07-16 22:45:32,0x6651a0a95e7e19c13dd94cab16c91c201337b56a,Fail,3272713877515929381,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.769223
0xd16a46283e4a2d696f9c5f33f979f1f6b742dee8f96f8d95a298860099f71375,mainnet,2019-07-25 10:17:39,0xd7fe300587d41ed0e8b6a2bed5a1b2bb4fcdad9e,Fail,1830223736408604446481,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.436851
0xe0502149219a4071d10c81ed3ac75602602333d5279f8b5b9c1d9b95013a75b9,mainnet,2019-07-18 21:14:38,0xd65478656497b3388c2c930de3bc48ac0688039d,Fail,117751629644545480084,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.440323
0xe638fe0187bf3ab3e94b64d4b94de090533839436ebe767db4372bb51f00c366,mainnet,2019-07-14 22:29:44,0xe858a4bf603995a9156edbd25ff06269d997839e,Fail,26212607727907855277820,0x519b70055af55a007110b4ff99b0ea33071c720a,0xb92d2df99a47244c07a9d7ef73530c273f1d65230dbf...,0.694151
